In [1]:
import pandas as pd
import tweepy
from tqdm.notebook import tqdm
import pickle

## Load data 

In [2]:
path_train='labelled_train_data.csv'
df_train=pd.read_csv(path_train)

path_test='labelled_test_data.csv'
df_test=pd.read_csv(path_test)


## Get tweets by id 

In [3]:
## define twitter API 

consumer_key = "xxxxxxxx" 
consumer_secret = "xxxxxxxxxx" 
access_token = "xxxxxxxxxxxxxxxxx" 
access_token_secret = "xxxxxxxx" 

# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 

# set access to user's access key and access secret  
auth.set_access_token(access_token, access_token_secret) 
  
# calling the api  
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True) 

In [8]:
def get_text(id_):
    try :
        tweet=api.get_status(id_)
        return tweet.text
    except :
        return None

tab_texts=[]
for index,rows in tqdm(df_test.iterrows(),total=len(df_test)) :
    tab_texts.append(get_text(rows['id']))
    if index%100==0 :
        with open('/home/hamza/projects/filiere/saved_data/text_sent_test.pkl','wb') as pkl :
            pickle.dump(tab_texts,pkl)
with open('/home/hamza/projects/filiere/saved_data/text_sent_test.pkl','wb') as pkl :
            pickle.dump(tab_texts,pkl)

Rate limit reached. Sleeping for: 301
Rate limit reached. Sleeping for: 657
Rate limit reached. Sleeping for: 663


## add to csv and remove Nones 

In [3]:
tab_texts_train=pickle.load(open('/home/hamza/projects/filiere/saved_data/text_sent_all.pkl','rb'))
tab_texts_test=pickle.load(open('/home/hamza/projects/filiere/saved_data/text_sent_test.pkl','rb'))

df_train['text']=tab_texts_train
df_test['text']=tab_texts_test

# Merge

df_all=pd.concat([df_train,df_test], ignore_index=True)
#print(len(df_all))
df_all.dropna(subset=['text'], inplace=True)

## categorize polarity 
df_all.polarity=pd.Categorical(df_all.polarity)
df_all.rename(columns={'polarity':'pol_symbol'},inplace=True)

dict_map={'=':2,'-':0,'+':1}
df_all['polarity']=df_all.pol_symbol.map(dict_map)
del df_all['Unnamed: 0']

## Preprocess and clean 

In [5]:
from utils import clean 
df_all['text']=df_all.text.apply(lambda x:clean.clean(x))


In [7]:
### separate train test 
from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(df_all, test_size=0.2)

In [12]:
trainset

,id,pol_symbol,nature,emotion,text,polarity
0,487830913032519681,=,INFORMATION,NaN,Et pendant ce temps la elles tournent via,2
1,505675738155347968,=,INFORMATION,NaN,. vous oubliez l ' abbé Pierre responsabl...,2
2,519392800002297857,+,OPINION,VALORISATION,[ LoiRoyal ] le solaire : l ' énergie...,1
3,506828044732878848,=,INFORMATION,NaN,"คันหู วาเลนไทน์ 2010 : via OUI , le récha...",2
4,488949864600829953,=,INFORMATION,NaN,Correze - MAISON A OSSATURE BOIS ECOLOGIQUE :...,2
...,...,...,...,...,...,...
7043,520252139172810753,+,OPINION,VALORISATION,Et une très bonne nouvelle pour les Français p...,1
7044,489438791703470080,-,SENTIMENT,NaN,Biodiversité et climat : Deux crises qui gl...,0
7045,507083059359789056,=,INFORMATION,NaN,"ecologie « La Constellation du chien », roman ...",2
7046,518726125501956096,=,INFORMATION,NaN,Manif pour tous : le sénateur écologiste J...,2


In [10]:
trainset.reset_index(inplace=True,drop=True)
testset.reset_index(inplace=True,drop=True)

trainset.to_csv('/home/hamza/projects/filiere/etude_de_cas/data/trainset_sent.csv',index=False)
testset.to_csv('/home/hamza/projects/filiere/etude_de_cas/data/testset_sent.csv',index=False)